In [ ]:
import pandas as pd
import numpy as np
import json
from flatten_json import flatten
import pickle
from hx_lib import *
from hx_tools import *
user_id = USER_ID
pwd = API_KEY

In [ ]:
def hx_get_hosts(hx):
    # Use either aws or local to signify the appliance to be used.
    if hx == 'aws':
        hx_appliance = AWS_SVR
    elif hx == 'local':
        hx_appliance = LOCAL_SVR
    # Then makes a call to an underlying class (HXAPI) to log into the designated appliance.
    hx_api_object = HXAPI(hx_appliance, hx_port = 3000)
    (ret, response_code, response_data) = hx_api_object.restLogin(user_id, pwd)
    hresponse_data = {}
    # If the login is successful, get records for all agents. In this case, with an upper limit of 150k records.
    if ret:
        (ret, hresponse_code, hresponse_data) = hx_api_object.restListHosts(limit=150000)
        return ret, hresponse_code, hresponse_data

In [ ]:
# Makes API call to stores results in 'results' variable
results = hx_get_hosts(hx='eag')

In [ ]:
# Create variable to store just the actual records and not the extraneous information returned in the original API results.
data = (results[2]['data']['entries'])

In [ ]:
# Create a DataFrame from the returned API results
hosts_df = pd.json_normalize(data, max_level=1)

In [ ]:
# Verify column names of the DataFrame
print(list(hosts_df))

In [ ]:
# Display DataFrame
hosts_df

## Agent Version Pivot


In [ ]:
# Create a pivot table to divide agents into OS type then to display the agent versions, counts for each version, and a total agent count.
agent_pivot = pd.pivot_table(hosts_df.fillna(0), index=['os.platform', 'agent_version'], aggfunc='count', margins = True)
agent_df = agent_pivot['_id']
agent_df


## Agent OS Pivot

In [ ]:
# Create a pivot table to divide agents into OS type and then into OS versions and counts of each with a total.
pd.options.display.max_rows = 100
os_pivot = pd.pivot_table(hosts_df.fillna(0), index=['os.platform', 'os.product_name'], aggfunc='count', margins = True)
os_df = os_pivot['_id']
os_df

In [ ]:
# Create Excel document with a tab for the DataFrame and a tab for the Pivot Table
writer = pd.ExcelWriter('FireEye OS Breakdown.xlsx', engine='xlsxwriter')
agent_df.to_excel(writer, 'OS Pivot Table')
writer.save()
writer.close()

In [ ]:
# Create a DataFrame with the OS as the index
agent_df = agent_df.to_frame()
agent_df_indexed = agent_df.reset_index(level=['os.product_name'])
agent_df_indexed

# storing agent pivot table permanently
agent_df_currentdate = agent_df
%store agent_df_currentdate

In [ ]:
# Load a pickle file that was previously saved
fireeye_df_previousdate = pickle.load( open( "fireeye_df_previousdate.p", "rb" ) )

In [ ]:
# Display DataFrame
fireeye_df_previousdate

In [ ]:
# Sort agent records in DataFrame by date of last agent checkin, oldest to newest
sorted_fireeye_previousdate = fireeye_df_previousdate.sort_values(by='LAST_POLL_TIMESTAMP', ascending=False)
sorted_fireeye_previousdate

In [ ]:
# Deduplicate the agents, only keeping the oldest record
dedupe_fe_currentdate = sorted_fireeye_currentdate.drop_duplicates(subset=['HOSTNAME'], keep='first')
dedupe_fe_currentdate

In [ ]:
# Create a pivot table to divide agents into OS type then to display the agent versions, counts for each version, and a total agent count.
agent_pivot_currentdate = pd.pivot_table(dedupe_fe_currentdate.fillna(0), index=['os.platform', 'AGENT_VERSION'], aggfunc='count', margins = True)
agent_df_currentdate = agent_pivot_currentdate['_ID']
agent_df_currentdate

In [ ]:
# Reset the index of the DataFrame and create new variable to store these changes.
agent_df_currentdate_indexed = agent_df_currentdate.reset_index(level='AGENT_VERSION')
agent_df_currentdate_indexed

In [ ]:
# print(type(agent_df_currentdate))
# test_df = test_df.to_frame()
agent_df_currentdate_indexed['AGENT_VERSION'] == agent_df_previousdate_indexed['AGENT_VERSION']

In [ ]:
# Get shape of the current DataFrame
agent_df_currentdate_indexed.shape

In [ ]:
#  Get shape of previous DataFrame to compare with current DataFrame for a sanity check
agent_df_previousdate_indexed.shape

In [ ]:
agent_df_currentdate_indexed.compare(agent_df_previousdate_indexed, keep_equal=True, keep_shape=True)

In [ ]:
agent_df_currentdate_comparison = agent_df_currentdate_indexed

In [ ]:
agent_df_currentdate_comparison['Current Count'] = agent_df_previousdate_indexed['_ID']

In [ ]:
# Prep data to be graphed
agent_graph_ready = agent_df_currentdate_indexed_test
agent_graph_ready = agent_graph_ready.iloc[:-1 , :]
# agent_graph_ready = agent_graph_ready.drop(columns=['Delta'])
agent_graph_ready = agent_graph_ready.rename(columns={"_ID": "1-5-2022 Count"})
agent_graph_ready= agent_graph_ready.reset_index()
agent_graph_ready = agent_graph_ready.set_index(['os.platform', 'AGENT_VERSION'])
agent_graph_ready

In [ ]:
# Create graph
agent_graph_ready.plot(figsize=(20,10),logy=True, kind='bar')


In [ ]:
# Combine the previous data with the current data and show the change (Delta column) between the 2 dates
agent_df_currentdate_indexed_test['Delta'] = agent_df_currentdate_indexed_test['_ID'].sub(agent_df_currentdate_indexed_test['1-10-2020 Count'], axis = 0)
agent_df_currentdate_indexed_test

In [ ]:
# Create an Excel document and create a tab for each of the DataFrames created
writer = pd.ExcelWriter('FireEye Agent Version Breakdown.xlsx', engine='xlsxwriter')
raw_pivot.to_excel(writer, 'Pivot Tables', startcol=0)
mac_pivot.to_excel(writer, 'Pivot Tables', startcol=4)
agent_df_currentdate.to_excel(writer, '1-5-2023 Agent Breakdown')
agent_df_01102022.to_excel(writer, '1-10-2023 Agent Breakdown')
agent_df_01312022.to_excel(writer, '1-10-2023 Agent Breakdown')
linux_agents.to_excel(writer, 'Linux Servers', index=False)
win_servers.to_excel(writer, 'Windows Servers', index=False)
workstations.to_excel(writer, 'All Workstations', index=False)
win_workstations.to_excel(writer, 'Windows Workstations', index=False)
mac_workstations.to_excel(writer, 'Mac Workstations', index=False)
cloud_agents.to_excel(writer, 'All Cloud Servers', index=False)
cloud_windows.to_excel(writer, 'Cloud Windows Servers', index=False)
cloud_linux.to_excel(writer, 'Cloud Linux Servers', index=False)
df.to_excel(writer, 'Raw Data', index=False)
writer.save()
writer.close()

## Old Agent Work Area

In [ ]:
# Create a list of agent versions that need to be upgraded
trouble_group = ['30.19.6', '31.28.4', '30.19.3', '31.28.4', '23.10.0', '26.21.10', '29.7.0', '30.19.3', '30.19.6', '30.19.8', '31.28.4', '31.28.8', '32.30.0', '32.30.10']

In [ ]:
# Scrub data so it can be consumed
trouble_agents = hosts_df[hosts_df['agent_version'].isin(trouble_group)]
trouble_agent_sorted = trouble_agents.sort_values(by=['agent_version'])
trouble_agent_trim = trouble_agent_sorted[['_id', 'agent_version', 'hostname', 'domain', 'os.product_name', 'primary_mac', 'timezone', 'initial_agent_checkin', 'last_poll_timestamp', 'stats.acqs', 'stats.alerts']]
trouble_agent_trim.drop_duplicates(subset=['hostname'], inplace=True)
trouble_agent_trim

In [ ]:
# Create pivot table of the trouble agent versions
pd.options.display.max_rows = 100
old_agentos_pivot = pd.pivot_table(trouble_agent_trim.fillna(0), index=['os.product_name', 'agent_version'], aggfunc='count', margins = True)
old_agentos_df = old_agentos_pivot['_id']
old_agentos_df

In [ ]:
# Create a pivot table showing the agent versions and count of the agents in each agent version
pd.options.display.max_rows = 100
old_agent_pivot = pd.pivot_table(trouble_agent_trim.fillna(0), index=['agent_version'], aggfunc='count', margins = True)
old_agent_df = old_agent_pivot['_id']
old_agent_df

In [ ]:
# Create pivot table showing the OS versions for the trouble agents and give a count per OS version
pd.options.display.max_rows = 100
old_os_pivot = pd.pivot_table(trouble_agent_trim.fillna(0), index=['os.product_name'], aggfunc='count', margins = True)
old_os_df = old_os_pivot['_id']
old_os_df

In [ ]:
# Create Excel document with Potive tables and the raw data, each on their own tab
writer = pd.ExcelWriter('FireEye Old Agent Version Breakdown 4-13.xlsx', engine='xlsxwriter')
old_os_df.to_excel(writer, 'Pivot Tables', startcol=0)
old_agentos_df.to_excel(writer, 'Pivot Tables', startcol=4)
trouble_agent_trim.to_excel(writer, 'Old Agent Dataset', index=False)
writer.save()
writer.close()

### Power BI and Bigfix Work Area

In [ ]:
# Ingest hosts present in Power BI reports
powerbi_hosts = pd.read_excel('data - date1.xlsx')

In [ ]:
# Create a new variable that contains hosts that are not found in the powerbi_bosts data
oldhosts_filtered = ~trouble_agent_trim['hostname'].isin(powerbi_hosts['SERVER_NAME'])
oldhosts_nopbi = trouble_agent_trim[oldhosts_filtered]
oldhosts_nopbi

In [ ]:
# Create Excel doc containing all hosts that do not show up in Power BI reporting
oldhosts_nopbi.to_excel('FireEye Old Agents No PBI.xlsx', index=False)

In [ ]:
# Create pivot table with the OS versions for the hosts with the oldest agent check in times with counts for each OS
pd.options.display.max_rows = 100
old_os_pivot = pd.pivot_table(oldhosts_nopbi.fillna(0), index=['os.product_name'], aggfunc='count', margins = True)
old_os_df = old_os_pivot['_id']
old_os_df

In [ ]:
# Create variable showing all the hosts that are in the Power BI data as well as the trouble agent data
oldhosts_filtered = powerbi_hosts['SERVER_NAME'].str.upper().isin(trouble_agent_trim['hostname'].str.upper())
oldhosts_enriched = powerbi_hosts[oldhosts_filtered]
oldhosts_enriched

In [ ]:
# Merge the trouble agent and Power BI data
pbi_merged = pd.merge(trouble_agent_trim, powerbi_hosts[['SERVER_NAME', 'AIID', 'APPLICATION_NAME','SUPPORT_GROUP', 'CONFIG_ADMIN_GROUP', 'U_SUPPORT_GROUP_L2', 'U_SUPPORT_GROUP_L3', 'RESOURCE_GROUP']],
                     left_on='hostname', right_on='SERVER_NAME', how='left')
pbi_merged

In [ ]:
# Deduplicate merged data
pbi_merged.drop_duplicates(subset=['hostname'],inplace=True)
pbi_merged

In [ ]:
# Merge the merged dataset with BigFix data
bf_merged = pd.merge(pbi_merged, bigfix_agents_df[['HOSTNAME','PLATFORM_SUPPORT', 'MACADDRESS']],
                     left_on=('hostname'), right_on=('HOSTNAME'), how='left')
bf_merged

In [ ]:
# Drop duplicates for the BigFix merged data
bf_merged.drop_duplicates(subset=['hostname'],inplace=True)
bf_merged

In [ ]:
# Create Excel doc out of BigFix curated DataFrame
bf_merged.to_excel('FireEye merge.xlsx', index=False)

In [ ]:
# Create DataFrame from CSV
bigfix_agents_df = pd.read_csv('BigFix-currentdate.csv')
bigfix_agents_df

In [ ]:
# Get DataFrame size
trouble_agent_trim.shape

## Delete Stale Agents

In [ ]:
stale_agents = []
stale_agents = pd.read_csv('stale_results.csv')

In [ ]:
hx_appliance = LOCAL_APPLIANCE
hx_api_object = HXAPI(hx_appliance, hx_port = 3000)
# Log into appliance
(ret, response_code, response_data) = hx_api_object.restLogin(user_id, pwd)
# If login successful, begin deletion process of each agent by ID
if ret:
    for agent in stale_agents:
        (d_ret, d_response_code, d_response_data) = hx_api_object.restDeleteHostByID(agent)
        if d_ret:
            print("[.] Deleted {}".format(agent))
        else:
            print(d_ret)
            print("[!] Failed to delete host {}. Error: {}".format(agent, d_response_data))

In [ ]:
# Designate hostset and get hosts in a list
stale_agent_hostset = hx_api_object.restListHostsInHostset('1111')

In [ ]:
# Create variable with just the hosts and no additional data that comes back 
data = (stale_agent_hostset[2]['data']['entries'])
data

In [ ]:
# Gather just the ID's from the returned data
stale_agent_ids = []
for agent in data:
    stale_agent_ids.append(agent['_id'])

In [ ]:
# Create pickle file of stale ids
pickle.dump( stale_agent_ids, open( "stale_agents_ids_currentdate.p", "wb" ) )

In [ ]:
# Create DataFrame from stale agent ids
stale_df = pd.DataFrame(stale_agent_ids)

In [ ]:
# Create CSV for stale agents
stale_df.to_csv('stale_agents_currentdate.csv',header=False, index=False)